In [12]:
from load_mnist import *
from KNNClassifier import *
from Dataset import *
import torch
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

a) K-Nearest Neighbors (kNN) Classifier

In [13]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [14]:
t_X_train, t_y_train = load_mnist('training',)
t_X_test, t_y_test = load_mnist('testing',)

In [15]:
rand_i = torch.randperm(len(t_y_train))[:1000]
rand_j = torch.randperm(len(t_y_test))[:1000]

In [16]:
t_X_train = t_X_train[rand_i, :, :]
t_y_train = t_y_train[rand_i]
t_X_test = t_X_test[rand_j, :, :]
t_y_test = t_y_test[rand_j]

In [17]:
def accuracy(y_test, y_pred):
    a = torch.sum(y_test == y_pred)
    b = len(y_test)
    accuracy = a.item() / b
    return accuracy

In [18]:
classifier = KNNClassifier(1, t_X_train, t_y_train)
y_pred = classifier.forward(t_X_test)
print("accuracy test1 for k = 1, ", accuracy(t_y_test, y_pred))

classifier2 = KNNClassifier(5, t_X_train, t_y_train)
y_pred2 = classifier2.forward(t_X_test)
print("accuracy test2 for k = 5, ", accuracy(t_y_test, y_pred2))

accuracy test1 for k = 1,  0.884
accuracy test2 for k = 5,  0.884


In [19]:
c_matrix = confusion_matrix(t_y_test, y_pred)
print(c_matrix)

c_matrix2 = confusion_matrix(t_y_test, y_pred2)
print(c_matrix2)

[[ 84   0   0   0   1   0   1   0   0   0]
 [  0 101   0   0   1   0   0   0   0   0]
 [  3   4  75   0   0   0   2   8   3   0]
 [  0   2   1 110   0   3   1   3   1   0]
 [  0   1   0   0  93   0   2   0   0   8]
 [  1   1   0   7   2  87   1   1   3   0]
 [  0   0   0   0   0   2  85   0   2   0]
 [  0   4   2   1   3   0   0 100   0   6]
 [  1   0   0   7   3   6   3   0  70   2]
 [  1   1   0   3   4   1   0   2   1  79]]
[[ 84   0   0   0   1   0   1   0   0   0]
 [  0 101   0   0   1   0   0   0   0   0]
 [  3   4  75   0   0   0   2   8   3   0]
 [  0   2   1 110   0   3   1   3   1   0]
 [  0   1   0   0  93   0   2   0   0   8]
 [  1   1   0   7   2  87   1   1   3   0]
 [  0   0   0   0   0   2  85   0   2   0]
 [  0   4   2   1   3   0   0 100   0   6]
 [  1   0   0   7   3   6   3   0  70   2]
 [  1   1   0   3   4   1   0   2   1  79]]


In [20]:
kfold = KFold(n_splits=5)

for train_index, test_index in kfold.split(t_X_train, t_y_train):  
    f_X_train = t_X_train[train_index] 
    f_y_train = t_y_train[train_index] 
    f_X_test = t_X_test[test_index] 
    f_y_test = t_y_test[test_index] 
    break 

b) Dataloading and Preprocessing

In [21]:
dataset = Dataset(t_X_train, t_y_train)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, drop_last=False)

mean: tensor(31.8839)
std: tensor(79.5719)


In [25]:
for epoch in range(5):
    for local_batch, local_labels in dataloader:
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
print(str('local_batch: ') + str(local_batch))
print(str('local_labels: ') + str(local_labels))

local_batch: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         

c) Linear Classifier with Logistic Function